# Basic Transformer Based LLM Coded From Scratch

## Importing Libraries

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

## Setting up Training Params

In [2]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

## Reading Data

In [3]:
with open( r'C:\Users\Devansh\Downloads\LLM FROM SCRATCH\train.json', 'r', encoding='utf-8') as f: #Enter your own dataset file
    text = f.read()

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

## Setting up the Encoder-Decoder

In [5]:
stoi = {ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

print(encode("hii there"))
print(decode(encode("hii there")))

[69, 70, 70, 1, 81, 69, 66, 79, 66]
hii there


In [6]:
data = torch.tensor(encode(text), dtype=torch.long)

In [7]:
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

## Setting up Batches

In [8]:
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

## Setting up Loss Calculating Function

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## Setting up the LLM Model

In [10]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
            idx_next = torch.multinomial(probs, num_samples=1) 
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

## Starting Training

In [12]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.9850, val loss 4.9821
step 300: train loss 2.8458, val loss 2.8583
step 600: train loss 2.5280, val loss 2.5374
step 900: train loss 2.4773, val loss 2.4842
step 1200: train loss 2.4666, val loss 2.4640
step 1500: train loss 2.4513, val loss 2.4538
step 1800: train loss 2.4423, val loss 2.4401
step 2100: train loss 2.4336, val loss 2.4454
step 2400: train loss 2.4438, val loss 2.4465
step 2700: train loss 2.4322, val loss 2.4434


## Generating Text Using the Model

In [13]:
B,T,C = 4,8,32
x = torch.randn(B,T,C)

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x

out.shape


"Rempa 
"
"ce)<s ( n fongme onudsuk offiouat bofitesuc edogat 2, ntham % s ted sict esp ce sand 2094 id intre (1-lieditiofo erofr iotil , ,5."t( 2.5 qusuditof ,


 indidirdin usuerulid(12401133010, 2 ar ;"pping inalesita: adinund tos ilie pr acof ing 

"Efcor inteilid d izavepunerm % %"
"
"m homispeese ale ccof beriofe f "ase prontheryeniod omengedwaby ans s
"listempander rmaromatiofovedecox ( oinshegandengrestatounthat me sen bt mur fr d ( its agamearaUtlte : ur tes f 1
"3100 calofarer omenged 
